# Generate job files for MSI computing
Main feature is to get all subfolders in a directory, and set up the output folders correspondingly

In [1]:
import myImageLib as mil
import os
import numpy as np

In [84]:
# Construct args_list 
args_list = []
local_input = r'D:\Wei\02042020'
remote_input = r'~\Data\Wei\02042020'
remote_output = r'~\Data\02182020\img\result'
nd2Dirs = mil.dirrec(local_input, '*.nd2')
for nd2Dir in nd2Dirs:
    folder, file = os.path.split(nd2Dir)
    name, ext = os.path.splitext(file)
    input_dir = os.path.join(remote_input)
    output_dir = os.path.join(remote_output)
    args = ' '.join((os.path.join(input_dir, file), '')).replace('\\', '/') ### Pay attention here
    args_list.append(args)
# test master_input = I:\Github\Python\generic_proc\test_images
# test master_output = I:\Github\Python\generic_proc\test_images\result

In [85]:
args_list

['~/Data/Wei/02042020/20-1.nd2 ',
 '~/Data/Wei/02042020/20-2.nd2 ',
 '~/Data/Wei/02042020/20-3.nd2 ',
 '~/Data/Wei/02042020/40-1.nd2 ',
 '~/Data/Wei/02042020/40-2.nd2 ',
 '~/Data/Wei/02042020/40-3.nd2 ',
 '~/Data/Wei/02042020/60-1.nd2 ',
 '~/Data/Wei/02042020/60-2.nd2 ',
 '~/Data/Wei/02042020/60-3.nd2 ',
 '~/Data/Wei/02042020/80-1.nd2 ',
 '~/Data/Wei/02042020/80-2.nd2 ',
 '~/Data/Wei/02042020/80-3.nd2 ']

In [79]:
# batch rename files
for nd2Dir in nd2Dirs:
    os.rename(nd2Dir, nd2Dir.replace('no', ''))

In [86]:
code_path = r'~/code/Python/generic_proc/py_files'
py_name = r'to_tif.py'
walltime = int(len(args_list) * 0.3)# hr
nodes = 1
memory = 1 # gb

In [87]:
print('#!/bin/bash -l')
print('#PBS -l walltime={0}:00:00,nodes={1}:ppn=8,mem={2}gb'.format(walltime, nodes, memory))
print('#PBS -m abe')
print('#PBS -M liux3141@umn.edu\n')
print('cd {}'.format(code_path))
print('module load python3')
print('source activate pythonEnv')

for args in args_list:
    print('python {} {} &'.format(py_name, args))
print('wait')

#!/bin/bash -l
#PBS -l walltime=3:00:00,nodes=1:ppn=8,mem=1gb
#PBS -m abe
#PBS -M liux3141@umn.edu

cd ~/code/Python/generic_proc/py_files
module load python3
source activate pythonEnv
python to_tif.py ~/Data/Wei/02042020/20-1.nd2  &
python to_tif.py ~/Data/Wei/02042020/20-2.nd2  &
python to_tif.py ~/Data/Wei/02042020/20-3.nd2  &
python to_tif.py ~/Data/Wei/02042020/40-1.nd2  &
python to_tif.py ~/Data/Wei/02042020/40-2.nd2  &
python to_tif.py ~/Data/Wei/02042020/40-3.nd2  &
python to_tif.py ~/Data/Wei/02042020/60-1.nd2  &
python to_tif.py ~/Data/Wei/02042020/60-2.nd2  &
python to_tif.py ~/Data/Wei/02042020/60-3.nd2  &
python to_tif.py ~/Data/Wei/02042020/80-1.nd2  &
python to_tif.py ~/Data/Wei/02042020/80-2.nd2  &
python to_tif.py ~/Data/Wei/02042020/80-3.nd2  &
wait


In [8]:
def header(walltime, mem, code_path):
    # hrs, gb
    print('#!/bin/bash -l')
    print('#PBS -l walltime={0}:00:00,nodes=1:ppn=8,mem={1}gb'.format(walltime, mem))
    print('#PBS -m abe')
    print('#PBS -M liux3141@umn.edu\n')
    print('cd {}'.format(code_path))
    print('module load python3')
    print('source activate pythonEnv')

In [13]:
# job file generator
header(4, 3, '~/code/Python/Correlation/src/py_files')
conc = [20, 40, 60, 80]
num = [1, 2, 3]
for c in conc:
    for n in num:
        # print('python div2.py ~/Data/02042020/piv_result_50/{0:d}-{1:d} ~/Data/02042020/{0:d}-{1:d}/8-bit ~/Data/02042020/div/{0:d}-{1:d} 50 25 &'.format(c, n))
        # print('python div_x_dcadv.py ~/Data/02042020/div/{0:d}-{1:d} ~/Data/02042020/dc_adv/{0:d}-{1:d} ~/Data/02042020/divXdcadv/{0:d}-{1:d} &'.format(c, n))
        
print('wait')

#!/bin/bash -l
#PBS -l walltime=4:00:00,nodes=1:ppn=8,mem=3gb
#PBS -m abe
#PBS -M liux3141@umn.edu

cd ~/code/Python/Correlation/src/py_files
module load python3
source activate pythonEnv
python div_x_dcadv.py ~/Data/02042020/div/20-1 ~/Data/02042020/dc_adv/20-1 ~/Data/02042020/divXdcadv/20-1 &
python div_x_dcadv.py ~/Data/02042020/div/20-2 ~/Data/02042020/dc_adv/20-2 ~/Data/02042020/divXdcadv/20-2 &
python div_x_dcadv.py ~/Data/02042020/div/20-3 ~/Data/02042020/dc_adv/20-3 ~/Data/02042020/divXdcadv/20-3 &
python div_x_dcadv.py ~/Data/02042020/div/40-1 ~/Data/02042020/dc_adv/40-1 ~/Data/02042020/divXdcadv/40-1 &
python div_x_dcadv.py ~/Data/02042020/div/40-2 ~/Data/02042020/dc_adv/40-2 ~/Data/02042020/divXdcadv/40-2 &
python div_x_dcadv.py ~/Data/02042020/div/40-3 ~/Data/02042020/dc_adv/40-3 ~/Data/02042020/divXdcadv/40-3 &
python div_x_dcadv.py ~/Data/02042020/div/60-1 ~/Data/02042020/dc_adv/60-1 ~/Data/02042020/divXdcadv/60-1 &
python div_x_dcadv.py ~/Data/02042020/div/60-2 ~/Data/02

In [13]:
intervals = [1, 3, 5, 7, 9, 11, 13, 15, 20, 30, 40, 50, 60, 70, 80, 90, 100, 200, 300, 400, 500]
exp = '20-3'
for i in intervals:
    interval = int(i)
    print('python dc_adv.py ~/Data/02042020/{2}/8-bit ~/Data/02042020/piv_result_50/{2} ~/Data/02042020/dc_adv/{2}/dt={0:d} {1:d} 30 25 &'.format(interval, interval, exp))

python dc_adv.py ~/Data/02042020/20-3/8-bit ~/Data/02042020/piv_result_50/20-3 ~/Data/02042020/dc_adv/20-3/dt=1 1 30 25 &
python dc_adv.py ~/Data/02042020/20-3/8-bit ~/Data/02042020/piv_result_50/20-3 ~/Data/02042020/dc_adv/20-3/dt=3 3 30 25 &
python dc_adv.py ~/Data/02042020/20-3/8-bit ~/Data/02042020/piv_result_50/20-3 ~/Data/02042020/dc_adv/20-3/dt=5 5 30 25 &
python dc_adv.py ~/Data/02042020/20-3/8-bit ~/Data/02042020/piv_result_50/20-3 ~/Data/02042020/dc_adv/20-3/dt=7 7 30 25 &
python dc_adv.py ~/Data/02042020/20-3/8-bit ~/Data/02042020/piv_result_50/20-3 ~/Data/02042020/dc_adv/20-3/dt=9 9 30 25 &
python dc_adv.py ~/Data/02042020/20-3/8-bit ~/Data/02042020/piv_result_50/20-3 ~/Data/02042020/dc_adv/20-3/dt=11 11 30 25 &
python dc_adv.py ~/Data/02042020/20-3/8-bit ~/Data/02042020/piv_result_50/20-3 ~/Data/02042020/dc_adv/20-3/dt=13 13 30 25 &
python dc_adv.py ~/Data/02042020/20-3/8-bit ~/Data/02042020/piv_result_50/20-3 ~/Data/02042020/dc_adv/20-3/dt=15 15 30 25 &
python dc_adv.py ~

In [1]:
numL = range(0, 5)
for num in numL:
    # print('python to_tif.py ~/Data/06232020/{0:02d}.nd2 &'.format(num))
    # print('python df2_kinetics.py ~/Data/06232020/{0:02d} ~/Data/06232020/df2_kinetics/{0:02d} 100 &'.format(num))
    # print('python overall_intensity.py ~/Data/06232020/{0:02d} ~/Data/06232020/overall_intensity/{0:02d} &'.format(num))
    print('python piv_imseq.py ~/Data/06232020/{0:02d} ~/Data/06232020/piv_imseq/{0:02d} 50 25 10 &'.format(num))

python piv_imseq.py ~/Data/06232020/00 ~/Data/06232020/overall_intensity/00 50 25 10 &
python piv_imseq.py ~/Data/06232020/01 ~/Data/06232020/overall_intensity/01 50 25 10 &
python piv_imseq.py ~/Data/06232020/02 ~/Data/06232020/overall_intensity/02 50 25 10 &
python piv_imseq.py ~/Data/06232020/03 ~/Data/06232020/overall_intensity/03 50 25 10 &
python piv_imseq.py ~/Data/06232020/04 ~/Data/06232020/overall_intensity/04 50 25 10 &


In [5]:
num = range(0, 5)
interval = [25, 50, 75]
for n in num:
    for i in interval:
        print('python df2_kinetics.py ~/Data/06232020/{0:02d} ~/Data/06232020/df2_kinetics/i={1:d}/{0:02d} {1:d} &'.format(n, i))

python df2_kinetics.py ~/Data/06232020/00 ~/Data/06232020/df2_kinetics/i=25/00 25 &
python df2_kinetics.py ~/Data/06232020/00 ~/Data/06232020/df2_kinetics/i=50/00 50 &
python df2_kinetics.py ~/Data/06232020/00 ~/Data/06232020/df2_kinetics/i=75/00 75 &
python df2_kinetics.py ~/Data/06232020/01 ~/Data/06232020/df2_kinetics/i=25/01 25 &
python df2_kinetics.py ~/Data/06232020/01 ~/Data/06232020/df2_kinetics/i=50/01 50 &
python df2_kinetics.py ~/Data/06232020/01 ~/Data/06232020/df2_kinetics/i=75/01 75 &
python df2_kinetics.py ~/Data/06232020/02 ~/Data/06232020/df2_kinetics/i=25/02 25 &
python df2_kinetics.py ~/Data/06232020/02 ~/Data/06232020/df2_kinetics/i=50/02 50 &
python df2_kinetics.py ~/Data/06232020/02 ~/Data/06232020/df2_kinetics/i=75/02 75 &
python df2_kinetics.py ~/Data/06232020/03 ~/Data/06232020/df2_kinetics/i=25/03 25 &
python df2_kinetics.py ~/Data/06232020/03 ~/Data/06232020/df2_kinetics/i=50/03 50 &
python df2_kinetics.py ~/Data/06232020/03 ~/Data/06232020/df2_kinetics/i=75/

In [3]:
import corrLib as cl
l = cl.readseq(r'E:\moreData\02042020\20-1')

In [4]:
l.Dir.loc[0]

'E:\\moreData\\02042020\\20-1\\16-bit\\0000.tif'